# Gerekli kütüphaneleri ekliyoruz

In [2]:
import numpy as np
import pandas as pd
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, matthews_corrcoef

# Eğiteceğimiz CSV dosyasını okuyoruz

In [3]:
df = pd.read_csv("data.csv", sep="|")
#a = pd.read_csv(data_path, delimiter='|')
# a.dropna(subset=['text'], inplace=True)

# a["target"].unique()

# Farklı label'ları farklı sayılara çeviriyoruz

#a['labels'] = pd.factorize(a.target)[0]
df['labels'] = df['target'].map({"OTHER": 0, "SEXIST": 1, "RACIST": 2, "INSULT": 3, "PROFANITY": 4})


# Sadece "text" ve "label" sütunlarını alarak veri kümemizi sadeleştiriyoruz

In [4]:
train, test = train_test_split(df, test_size=0.1, random_state=42)

train = train[["text", "labels"]]
test = test[["text", "labels"]]

train["text"] = train["text"].apply(lambda r: str(r))
train['labels'] = train['labels'].astype(int)

# Modeli tanımlıyoruz

In [5]:
model = ClassificationModel('bert', 'dbmdz/bert-base-turkish-uncased', num_labels=5, use_cuda=False,
                            args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 3, "train_batch_size": 64 , "fp16":False,"output_dir":"bert_model"})

C:\Users\Gazi\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

C:\Users\Gazi\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Gazi\.cache\huggingface\hub\models--dbmdz--bert-base-turkish-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

In [6]:
def compute_metrics(preds, labels):
    assert len(preds) == len(labels)
    f1 = f1_score(labels, preds, average='weighted')  # 'weighted' kullanıldı
    acc = accuracy_score(labels, preds)
    mcc = matthews_corrcoef(labels, preds)
    return {
        "f1": f1,
        "acc": acc,
        "mcc": mcc,
    }

# Modeli Eğitiyoruz

In [ ]:
model.train_model(train)

# Modeli değerlendirme
result, model_outputs, wrong_predictions = model.eval_model(test, compute_metrics=compute_metrics)

# Tahminleri elde etme
predictions = np.argmax(model_outputs, axis=1)
actuals = test.labels.values

print("Sonuçlar:", result)
print("Tahminler:", predictions)
print("Gerçekler:", actuals)


model_name = "turkish_bert_model"
model_path = f"{model_name}"
model.save_model(model_path)

  0%|          | 0/22 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/176 [00:00<?, ?it/s]